# Bibliotecas

In [5]:
import requests
import json
import pandas as pd
import pyarrow
from time import sleep, time
import datetime
from multiprocessing.pool import ThreadPool

# Requisição API

In [6]:
jql_str =''' 
            Status != Cancelado
            AND issuetype in (bug,story,"tech value",toil, opportunity,Hypothesis)
            AND created >= 2022-06-01 AND created <= -0d
            AND category in ("BU_ZAP+",BU_OLX,CROSS)
            order by created DESC'''

In [7]:
start = time()

In [8]:
from jira import JIRA
import pandas as pd
import numpy as np

jiraOptions = {'server': "https://olxbr.atlassian.net"}
  
jira = JIRA(options=jiraOptions, basic_auth=(
    "Your company mail", "Your key to acess Jira's API"))

data = jira.search_issues(jql_str=jql_str,maxResults = False, json_result=False)
# list_issues = data['issues']

In [9]:
end = time()
total_time = end-start
print(f'Tempo de espera: {round(total_time/60)} minutos')

Tempo de espera: 4 minutos


# Função 

In [10]:
from joblib import Parallel, delayed


def get_issue(issue):
    issues = []
    issues.append(issue.id + ";" 
                   + issue.fields.status.name + ";"
                   + issue.key + ";"
#                    + issue.fields.creator.displayName + ";"
                   + issue.fields.created + ";"
                   + str(issue.fields.resolutiondate) + ";"
                   + issue.fields.issuetype.name + ";"
                   + issue.fields.project.name + ";"
                   + issue.fields.project.key)
    return issues

total_issues = Parallel(n_jobs=3)(delayed(get_issue)(issue,) for issue in data)

#### Desempacote das listas

In [11]:
issues = ['issue_id;issue_status;issue_key;created_date;resolution_date;issue_type;name_project;key_project']
for n in range(len(total_issues)):
    issues.append(total_issues[n][0])

In [12]:
textfile = open("dados_jira.txt", "w",encoding='utf-8')
for element in issues:
    textfile. write(element + "\n")
textfile.close()

In [13]:
df = pd.read_csv('dados_jira.txt',sep=";")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   issue_id         3755 non-null   int64 
 1   issue_status     3755 non-null   object
 2   issue_key        3755 non-null   object
 3   created_date     3755 non-null   object
 4   resolution_date  3755 non-null   object
 5   issue_type       3755 non-null   object
 6   name_project     3755 non-null   object
 7   key_project      3755 non-null   object
dtypes: int64(1), object(7)
memory usage: 234.8+ KB


In [16]:
df['created_date'] = pd.to_datetime(df['created_date']).dt.date

In [18]:
df.to_csv('jira_issue.csv',index=False)

In [17]:
df.head(3)

,issue_id,issue_status,issue_key,created_date,resolution_date,issue_type,name_project,key_project
0,129363,Inbox,ANV-1748,2022-07-18,None,Bug,ANAPRO,ANV
1,129362,Progress,VASP-193,2022-07-18,None,Hypothesis,VAS Premium,VASP
2,129361,Em andamento,OBS-564,2022-07-18,None,Tech Value,Observability,OBS


## Email's

#Gerando Arquivo .CSV

def get_email(issue):
    email = []
    try:
        email.append(issue.id + ";" + issue.fields.creator.emailAddress)

    except Exception:
        email.append(issue.id + ";Undefined")
    return email

total_email = Parallel(n_jobs=3)(delayed(get_email)(issue,) for issue in data)
        

# Desempacote das listas
email = ['Issue_id;Email']
for n in range(len(total_email)):
    email.append(total_email[n][0])

#Salvar os dados em um arquivo txt para facilitar a quebra em ; de cada coluna
textfile = open("dados_email.txt", "w",encoding='utf-8')
for element in email:
    textfile. write(element + "\n")
textfile.close()

df_email = pd.read_csv('dados_email.txt',sep=";")

df = pd.merge(df, df_email,how = 'left', on = 'Issue_id')

## BU's

Para extrair as BU's ou Categorias de serviço das issues, é necessário que se faça de forma separada para ignorar o laço quando essa categoria estiver nula(por conta que alguns times não cadastraram a categoria)

In [ ]:
#Gerando Arquivo .CSV

def get_bu(issue):
    BU = []
    try:
        BU.append(issue.id + ";" + issue.fields.project.projectCategory.name)

    except Exception:
        BU.append(issue.id + ";Undefined")
    return BU

total_BU = Parallel(n_jobs=3)(delayed(get_bu)(issue,) for issue in data)
        

In [ ]:
df.info()

In [ ]:
# Desempacote das listas
BU = ['issue_id;bu']
for n in range(len(total_BU)):
    BU.append(total_BU[n][0])

In [ ]:
#Salvar os dados em um arquivo txt para facilitar a quebra em ; de cada coluna
textfile = open("dados_bu.txt", "w",encoding='utf-8')
for element in BU:
    textfile. write(element + "\n")
textfile.close()

In [ ]:
df_bu = pd.read_csv('dados_bu.txt',sep=";")

In [ ]:
df = pd.merge(df, df_bu,how = 'left', on = 'issue_id')

In [ ]:
df.head()

## Prioridades das issues

Para extrair as prioridades das issues, é necessário que se faça de forma separada para ignorar o laço quando essa categoria estiver nula (por conta que alguns times não cadastraram a categoria)

In [ ]:
#Gerando Arquivo .CSV

def get_prioridade(issue):
    prioridade = []
    try:
        prioridade.append(issue.id + ";" + issue.fields.priority.name)

    except Exception:
        prioridade.append(issue.id + ";Undefined")
    return prioridade

total_prioridade = Parallel(n_jobs=3)(delayed(get_prioridade)(issue,) for issue in data)

In [ ]:
# Desempacote das listas
prioridade = ['issue_id;priority']
for n in range(len(total_prioridade)):
    prioridade.append(total_prioridade[n][0])

In [ ]:
#Salvar os dados em um arquivo txt para facilitar a quebra em ; de cada coluna
textfile = open("dados_prioridade.txt", "w",encoding='utf-8')
for element in prioridade:
    textfile. write(element + "\n")
textfile.close()

In [ ]:
df_prioridade = pd.read_csv('dados_prioridade.txt',sep=";")

In [ ]:
df = pd.merge(df, df_prioridade,how = 'left', on = 'issue_id')

# Resolution
Para extrair o resolution das issues, é necessário que se faça de forma separada para ignorar o laço quando essa categoria estiver nula (por conta que alguns times não cadastraram a categoria)

In [ ]:
#Gerando Arquivo .CSV

def get_resolution(issue):
    resolution = []
    try:
        resolution.append(issue.id + ";" + issue.fields.resolution.name)

    except Exception:
        resolution.append(issue.id + ";Undefined")
    return resolution

total_resolution = Parallel(n_jobs=3)(delayed(get_resolution)(issue,) for issue in data)

In [ ]:
# Desempacote das listas
resolution = ['issue_id;resolution']
for n in range(len(total_resolution)):
    resolution.append(total_resolution[n][0])

In [ ]:
#Salvar os dados em um arquivo txt para facilitar a quebra em ; de cada coluna
textfile = open("dados_resolution.txt", "w",encoding='utf-8')
for element in resolution:
    textfile. write(element + "\n")
textfile.close()

In [ ]:
df_resolution = pd.read_csv('dados_resolution.txt',sep=";")

In [ ]:
df = pd.merge(df, df_resolution,how = 'left', on = 'issue_id')

In [ ]:
df.head()

In [ ]:
df.info()

# Labels
Para extrair o Labels das issues, é necessário que se faça de forma separada para ignorar o laço quando este campo estiver nulo.

In [ ]:
#Gerando Arquivo .CSV

def get_labels(issue):
    labels = []
    id_labels = []
    try:
        labels.append(issue.fields.labels)
        id_labels.append(issue.id)
    except Exception:
        labels.append(issue.id + ";Undefined")
    
    #Junção de id's e labels    
    lista_bruta = list(zip(id_labels, labels))
    
    return lista_bruta

total_labels = Parallel(n_jobs=3)(delayed(get_labels)(issue,) for issue in data)

In [ ]:
total_labels[:1]

In [ ]:
new_label= []
new_id = []
labels = str()
for n in range(len(total_labels)):
    if len(total_labels[n][0][1]) > 0:
        new_id.append(total_labels[n][0][0])
        for c in range(len(total_labels[n][0][1])):
            labels += total_labels[n][0][1][c] + ','
        new_label.append(labels[:-1])
        labels = str()

lista_final = list(zip(new_id, new_label))

In [ ]:
df_labels = pd.DataFrame(lista_final, columns=['issue_id','labels'])
df_labels['issue_id'] = df_labels['issue_id'].values.astype(int)

In [ ]:
df = pd.merge(df, df_labels,how = 'left', on = 'issue_id')

In [ ]:
df.head(2)

# Linked issues
Para extrair os links das issues que são derivadas de outras, é necessário que se faça de forma separada para ignorar o laço quando esse link não existir

In [ ]:
df_links = pd.read_csv('links.csv')

In [ ]:
df_links.rename(columns={'issue_Id':'issue_id'},inplace=True)

In [ ]:
df = pd.merge(df, df_links,how = 'left', on = 'issue_id')

# Parent ID
Para extrair os ID's das issues que são derivadas de outras, é necessário que se faça de forma separada para ignorar o laço quando esse ID não existir (por conta que algumas issues são Head/Parent)

In [ ]:
def get_parents(issue):
    issues_parents = []
    try:
        issues_parents.append(issue.id + ";"
                              + str(issue.fields.parent.id))
                              
    except Exception:
        pass
    return issues_parents

total_parents = Parallel(n_jobs=3)(delayed(get_parents)(issue,) for issue in data)

In [ ]:
len(total_parents)

In [ ]:
# Desempacote das listas
issues_parents = ['issue_id;parent_id']
for n in range(len(total_parents)):
    if len(total_parents[n]) > 0:
        issues_parents.append(total_parents[n][0])
    else:
        pass

In [ ]:
#Salvar os dados em um arquivo txt para facilitar a quebra em ; de cada coluna
textfile = open("dados_parent_id.txt", "w",encoding='utf-8')
for element in issues_parents:
    textfile. write(element + "\n")
textfile.close()

In [ ]:
df_parent = pd.read_csv('dados_parent_id.txt',sep=";")

In [ ]:
df = pd.merge(df, df_parent,how = 'left', on = 'issue_id')

In [ ]:
df['parent_id'].fillna('0',inplace=True)

In [ ]:
df['parent_id'] = df['parent_id'].astype(int)

# Junção das datas de Change Category
Data Frame receberá importação de outro notebook onde foi coletado informações "datetime" de mundanças de categorias das issues durante os seus processos

In [ ]:
cycle_time = pd.read_csv('cycle_time.csv')

In [ ]:
df = pd.merge(df, cycle_time,how = 'left', on = 'issue_key')

In [ ]:
df.head()

Conversões de dados

In [ ]:
df['resolution_date'].replace('None','',inplace=True)

In [ ]:
df['created_date'] = pd.to_datetime(df['created_date']).dt.date
df['resolution_date'] = pd.to_datetime(df['resolution_date']).dt.date
df['selected_for_discovery'] = pd.to_datetime(df['selected_for_discovery']).dt.date
df['discovery_done'] = pd.to_datetime(df['discovery_done']).dt.date
df['in_progress'] = pd.to_datetime(df['in_progress']).dt.date
df['done'] = pd.to_datetime(df['done']).dt.date
dt = datetime.date.today()
df['dt'] = dt

import pandas as pd
import numpy as np
df = pd.read_csv('datalake.csv')

df = df[df['resolution_date'].notnull()]

In [55]:
df.drop(columns=['key_project'],inplace=True)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   issue_id                737 non-null    int64         
 1   issue_status            737 non-null    object        
 2   issue_key               737 non-null    object        
 3   created_date            737 non-null    object        
 4   resolution_date         183 non-null    object        
 5   issue_type              737 non-null    object        
 6   name_project            737 non-null    object        
 7   bu                      737 non-null    object        
 8   priority                737 non-null    object        
 9   resolution              737 non-null    object        
 10  labels                  163 non-null    object        
 11  links                   0 non-null      object        
 12  parent_id               737 non-null    int32     

In [57]:
df.head()

,issue_id,issue_status,issue_key,created_date,resolution_date,issue_type,name_project,bu,priority,resolution,labels,links,parent_id,selected_for_discovery,discovery_done,in_progress,done,dt
0,127489,New Item,TRACKEXP-1919,2022-07-13,NaT,Opportunity,Tracking & Experimentation,CROSS,Medium,Undefined,NaN,NaN,0,NaT,NaT,NaT,NaT,2022-07-13
1,127488,Inbox,ANV-1725,2022-07-13,NaT,História,ANAPRO,BU_ZAP+,Medium,Undefined,NaN,NaN,0,NaT,NaT,NaT,NaT,2022-07-13
2,127464,Backlog,DE-925,2022-07-13,NaT,Tech Value,Data Engineering Team,CROSS,Alta,Undefined,NaN,NaN,99063,NaT,NaT,NaT,NaT,2022-07-13
3,127463,Backlog,DE-924,2022-07-13,NaT,Tech Value,Data Engineering Team,CROSS,Alta,Undefined,NaN,NaN,99063,NaT,NaT,NaT,NaT,2022-07-13
4,127462,Backlog,DE-923,2022-07-13,NaT,História,Data Engineering Team,CROSS,Alta,Undefined,NaN,NaN,99063,NaT,NaT,NaT,NaT,2022-07-13


In [58]:
df.to_csv('jira_issue.csv',index=False)
df.to_parquet('jira_issue.parquet',index=False)